## THE PRODUCT 

### Libraries

In [419]:
#imports
import config
import spotipy
import pandas as pd
import json
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import re
import json
import numpy as np
import pprint
import pickle

%matplotlib inline

from sklearn import datasets # sklearn comes with some toy datasets to practice
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from IPython.display import IFrame
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, OneHotEncoder, OrdinalEncoder, Normalizer, RobustScaler, QuantileTransformer
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

### Model frame

In [ ]:
import random

while True:
    user_input = input("Enter the name of the song: ")
    found_match = False

    random_song = random.choice(top100["Titles"]) 
    
    for song in top100["Titles"]:
        if user_input.lower() in song.lower():
            print("------------------------------------")
            print(f"Based on your entry '{user_input}'")
            print(f"We would like to recommend you '{random_song}'")
            user_q = input("Would you like to try again? (YES/NO): ").lower()
            found_match = True
            break

    if found_match:
        if user_q == "no":
            print("You have now exited the program")
            break
    else:
        model_input = song_recommender(user_input)
        print("--------------------------------------------------------------------------")
        print(f"Based on your song our AI recommends you '{model_input}'.")

In [ ]:
while True:
    user_input = input("Enter the name of the song: ")
    found_match = False

    random_song = random.choice(top100["Titles"]) 
    
    for song in top100["Titles"]:
        if user_input.lower() in song.lower():
            print("------------------------------------")
            print(f"Based on your entry '{user_input}'")
            print(f"We would like to recommend you '{random_song}'")
            user_q = input("Would you like to try again? (YES/NO): ").lower()
            found_match = True
            break

    if found_match:
        if user_q == "no":
            print("You have now exited the program")
            break
    else:
        model_input = song_recommender(user_input)
        print("--------------------------------------------------------------------------")
        print(f"Based on your song our AI recommends you '{model_input}'.")

### The model

#### Functions

In [ ]:
from IPython.display import IFrame

def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

#### K-means 

##### Import and read df

In [402]:
df = pd.read_csv("concatenated_df")

In [403]:
columns_to_remove = ['id','type', 'uri', 'track_href', 'analysis_url']
df.drop(columns=columns_to_remove, inplace=True)

In [ ]:
df.info()

In [397]:
id_df = df.select_dtypes(include='object')
numerical_df = df.select_dtypes(include ='number')

##### Scale 

In [405]:
# scaled all numerical values but IDs
scaler = StandardScaler()
scaler.fit(df)
X_scaled = scaler.transform(df)
X_scaled_df = pd.DataFrame(X_scaled, columns = df.columns)

##### Train model 

In [406]:
kmeans = KMeans(n_clusters=10, random_state=1234)
kmeans.fit(X_scaled_df)

/Users/bart/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=10, random_state=1234)

In [254]:
labels = kmeans.labels_
labels

array([1, 6, 6, ..., 9, 0, 8], dtype=int32)

In [255]:
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [ ]:
clusters = kmeans.predict(X_scaled_df)
#clusters
pd.Series(clusters).value_counts().sort_index()

In [408]:
df['Clusters'] = clusters

In [409]:
df.head(1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Clusters
0,0.698,0.868,1,-9.104,1,0.0446,0.0395,0.0201,0.196,0.743,108.912,226513,4,1


#### User input

#### Song search 

In [521]:
user_input = input(f"Enter song:")
results = sp.search(q=user_input, limit=1)

song_id = results["tracks"]["items"][0]["id"]

play_song(song_id)

Enter song: billie jean - michael jackson


In [522]:
audio_features = sp.audio_features(song_id)

In [523]:
input_df = pd.DataFrame(audio_features)

In [524]:
# clean input_df. make it equal with df 
columns_to_remove = ['id','type', 'uri', 'track_href', 'analysis_url']
input_df.drop(columns=columns_to_remove, inplace=True)

#### Scale 

In [525]:
X_input_scaled = scaler.transform(input_df)
input_scaled_df = pd.DataFrame(X_input_scaled, columns = input_df.columns)

#### Predict cluster

In [526]:
input_cluster = kmeans.predict(input_scaled_df)

#clusters
pd.Series(input_cluster).value_counts().sort_index()

6    1
Name: count, dtype: int64

In [527]:
input_cluster_int = input_cluster[0]

In [528]:
input_combined_df['Clusters'] = input_cluster_int

#### Recommendation

In [529]:
selected_song = df_combined[df_combined['Clusters'] == input_cluster_int].sample()

In [530]:
predict_song_id = selected_song['id'].iloc[0]

In [531]:
play_song(predict_song_id)

In [520]:
play_song(song_id)